In [43]:
import os
import time
import pandas as pd
import utility as util
from db import DB
db = DB()

INTERVAL = 120

In [83]:
# print(db.get_daily_price('2379', True).head(1))
# print(db.get_daily_price('2474', True).head(1))
# print(db.get_daily_price('2610', True).head(1))

In [67]:
#####################
### create features
#####################
from talib import abstract

begin_time = time.time()
for stock_id in db.get_stock_info().index:
    if os.path.exists('ml_data/feature/' + stock_id + '.csv'):
        continue
    df_daily = db.get_daily_price(stock_id).loc['2013':].copy()
    df_daily['industry'] = db.get_stock_info().loc[stock_id, 'industry']
    df_daily['stock_id'] = int(stock_id)
    
    general_cols = ['industry', 'stock_id']
    
    ### fundamental
    df_monthly = db.get_by_stock_id(stock_id, 'monthly_revenue')
    df_quarterly = db.get_by_stock_id(stock_id, 'quarterly_report')

    def compared_ratio(s, num):
        return s / s.shift(num) - 1
    def compared_diff(s, num):
        return s - s.shift(num)

    df_monthly['3月營收'] = df_monthly.當月營收.rolling(3).mean()
    df_monthly['12月營收'] = df_monthly.當月營收.rolling(12).mean()
    df_monthly['當月營收_年增率'] = compared_ratio(df_monthly.當月營收, 12)
    df_monthly['3月營收_年增率'] = compared_ratio(df_monthly['3月營收'], 12)
    df_monthly['12月營收_年增率'] = compared_ratio(df_monthly['12月營收'], 12)
    count = 0
    df_monthly['當月營收_年增次數'] = 0
    for i in range(12, df_monthly.shape[0]):
        count = count + 1 if df_monthly.當月營收_年增率.iloc[i] > 0 else 0
        df_monthly.當月營收_年增次數.iloc[i] = count
    util.fill_short_interval_by_long_interval(df_daily, df_monthly, 
        ['當月營收_年增率', '3月營收_年增率', '12月營收_年增率', '當月營收_年增次數'])
        
    df_quarterly['毛利率'] = df_quarterly.毛利 / df_quarterly.營收
    df_quarterly['營益率'] = df_quarterly.營利 / df_quarterly.營收
    df_quarterly['淨利率'] = df_quarterly.稅後淨利 / df_quarterly.營收
    df_quarterly['毛利率_年增'] = compared_diff(df_quarterly.毛利率, 4)
    df_quarterly['營益率_年增'] = compared_diff(df_quarterly.營益率, 4)
    df_quarterly['淨利率_年增'] = compared_diff(df_quarterly.淨利率, 4)
    df_quarterly['毛利率_季增'] = compared_diff(df_quarterly.毛利率, 1)
    df_quarterly['營益率_季增'] = compared_diff(df_quarterly.營益率, 1)
    df_quarterly['淨利率_季增'] = compared_diff(df_quarterly.淨利率, 1)
    util.fill_short_interval_by_long_interval(df_daily, df_quarterly, 
        ['毛利率_年增', '營益率_年增', '淨利率_年增', '毛利率_季增', '營益率_季增', '淨利率_季增'])

    df_quarterly['EPS4季'] = df_quarterly.EPS.rolling(4).sum()
    df_quarterly['EPS4季_季增率'] = compared_ratio(df_quarterly.EPS4季, 1)
    df_quarterly['三率升降'] = (df_quarterly.毛利率_年增 > 0).apply(int) + (df_quarterly.營益率_年增 > 0).apply(int) \
                 + (df_quarterly.淨利率_年增 > 0).apply(int) + (df_quarterly.EPS4季_季增率 > 0).apply(int) - 2
    df_quarterly['三率升降累積'] = 0
    for i in range(5, df_quarterly.shape[0]):
        if df_quarterly.三率升降.iloc[i] > 0:
            if df_quarterly.三率升降累積.iloc[i - 1] > 0:
                df_quarterly.三率升降累積.iloc[i] = df_quarterly.三率升降累積.iloc[i - 1] + df_quarterly.三率升降.iloc[i]
            else:
                df_quarterly.三率升降累積.iloc[i] = df_quarterly.三率升降.iloc[i]
        elif df_quarterly.三率升降.iloc[i] < 0:
            if df_quarterly.三率升降累積.iloc[i - 1] < 0:
                df_quarterly.三率升降累積.iloc[i] = df_quarterly.三率升降累積.iloc[i - 1] + df_quarterly.三率升降.iloc[i]
            else:
                df_quarterly.三率升降累積.iloc[i] = df_quarterly.三率升降.iloc[i]
        else:
            df_quarterly.三率升降累積.iloc[i] = 0
    util.fill_short_interval_by_long_interval(df_daily, df_quarterly, 
        ['EPS4季', 'EPS4季_季增率', '三率升降', '三率升降累積'])
    
    股數 = df_quarterly.股本 / 10
    df_quarterly['每股淨值'] = df_quarterly.權益 / 股數
    util.fill_short_interval_by_long_interval(df_daily, df_quarterly, '每股淨值')

    df_daily['本益比'] = df_daily.close / df_daily.EPS4季
    df_daily['本淨比'] = df_daily.close / df_daily.每股淨值
    df_daily['眼光費'] = (df_daily.close - df_daily.每股淨值) / df_daily.EPS4季

    fundamental_cols = ['當月營收_年增率', '3月營收_年增率', '12月營收_年增率', '當月營收_年增次數', 
                        '毛利率_年增', '營益率_年增', '淨利率_年增', '毛利率_季增', '營益率_季增', '淨利率_季增', 
                        'EPS4季', 'EPS4季_季增率', '三率升降', '三率升降累積', 
                        '每股淨值', '本益比', '本淨比', '眼光費']

    ### technical
    df_daily['close_漲幅'] = compared_ratio(df_daily.close, 1)
    close_5 = df_daily.close.rolling(5).mean()
    close_10 = df_daily.close.rolling(10).mean()
    close_20 = df_daily.close.rolling(20).mean()
    close_60 = df_daily.close.rolling(60).mean()
    close_240 = df_daily.close.rolling(240).mean()
    volume_5 = df_daily.volume.rolling(5).mean()
    volume_10 = df_daily.volume.rolling(10).mean()
    df_daily['close_1-5'] = df_daily.close - close_5
    df_daily['close_5-10'] = close_5 - close_10
    df_daily['close_10-20'] = close_10 - close_20
    df_daily['close_1-10'] = df_daily.close - close_10
    df_daily['close_1-20'] = df_daily.close - close_20
    df_daily['close_1-60'] = df_daily.close - close_60
    df_daily['close_1-240'] = df_daily.close - close_240
    df_daily['volume_1-5'] = df_daily.volume - volume_5
    df_daily['volume_5-10'] = volume_5 - volume_10
    
    df_daily[['DIFF_日k', 'DEM_日k', 'D-M_日k']] = abstract.MACD(df_daily)
    df_daily[['K_日k', 'D_日k']] = abstract.STOCH(df_daily)
    df_daily['K-D_日k'] = df_daily.K_日k - df_daily.D_日k
    
    df_周k = util.combine_price(df_daily, 10)
    df_周k[['DIFF_周k', 'DEM_周k', 'D-M_周k']] = abstract.MACD(df_周k)
    df_周k[['K_周k', 'D_周k']] = abstract.STOCH(df_周k)
    df_周k['K-D_周k'] = df_周k.K_周k - df_周k.D_周k
    util.fill_short_interval_by_long_interval(df_daily, df_周k, ['DIFF_周k', 'DEM_周k', 'D-M_周k', 'K_周k', 'D_周k', 'K-D_周k'])

    df_月k = util.combine_price(df_daily, 20)
    df_月k[['DIFF_月k', 'DEM_月k', 'D-M_月k']] = abstract.MACD(df_月k)
    df_月k[['K_月k', 'D_月k']] = abstract.STOCH(df_月k)
    df_月k['K-D_月k'] = df_月k.K_月k - df_月k.D_月k
    util.fill_short_interval_by_long_interval(df_daily, df_月k, ['DIFF_月k', 'DEM_月k', 'D-M_月k', 'K_月k', 'D_月k', 'K-D_月k'])
        
    technical_cols = ['close', 'close_漲幅', 'close_1-5', 'close_5-10', 'close_10-20', 
                      'close_1-10', 'close_1-20', 'close_1-60', 'close_1-240', 
                      'volume_1-5', 'volume_5-10', 
                      'DIFF_日k', 'DEM_日k', 'D-M_日k', 'K_日k', 'D_日k', 'K-D_日k', 
                      'DIFF_周k', 'DEM_周k', 'D-M_周k', 'K_周k', 'D_周k', 'K-D_周k', 
                      'DIFF_月k', 'DEM_月k', 'D-M_月k', 'K_月k', 'D_月k', 'K-D_月k']

    ### chip
    buy_surplus = db.get_by_stock_id(stock_id, 'daily_buy_sell_surplus')
    df_daily = pd.merge(df_daily, buy_surplus, left_index=True, right_index=True)
    
    df_daily['foreign_5'] = df_daily.foreign.rolling(5).sum()
    df_daily['foreign_10'] = df_daily.foreign.rolling(10).sum()
    df_daily['foreign_20'] = df_daily.foreign.rolling(20).sum()
    df_daily['foreign_1-5'] = df_daily.foreign - df_daily.foreign_5
    df_daily['foreign_5-10'] = df_daily.foreign_5 - df_daily.foreign_10
    df_daily['foreign_10-20'] = df_daily.foreign_10 - df_daily.foreign_20

    chip_cols = ['foreign', 'foreign_5', 'foreign_10', 'foreign_20', 'foreign_1-5', 'foreign_5-10', 'foreign_10-20']

    df = df_daily[general_cols + fundamental_cols + technical_cols + chip_cols].dropna()
    df.to_csv('ml_data/feature/' + stock_id + '.csv', index=True, index_label='date')
    
end_time = time.time()
(end_time - begin_time) / 60

c:\users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


0.12548943360646567

In [68]:
#####################
### label 
#####################
def label(df):
    df = df.copy()
    max_close = df['close'].rolling(INTERVAL).max().shift(-INTERVAL)
    min_close = df['close'].rolling(INTERVAL).min().shift(-INTERVAL)
    max_close_ratio = max_close / df.close
    min_close_ratio = min_close / df.close
    
    df['label'] = 0
    df['weight'] = 1
    for i in range(len(df.index) - INTERVAL):
        if (max_close_ratio.iloc[i] > 1.1) & (min_close_ratio.iloc[i] > 0.95):
            df['label'].iloc[i] = 1
            df['weight'].iloc[i] = int((max_close_ratio.iloc[i] - 1) / 0.1)
        elif (min_close_ratio.iloc[i] < 0.9) & (max_close_ratio.iloc[i] < 1.05):
            df['label'].iloc[i] = -1
            df['weight'].iloc[i] = int((1 - min_close_ratio.iloc[i]) / 0.1)          
    return df

begin_time = time.time()

for stock_id in db.get_stock_info().index: 
    if os.path.exists('ml_data/label/' + stock_id + '.csv'):
        continue
    df = pd.read_csv('ml_data/feature/' + stock_id + '.csv', index_col=['date'], parse_dates=['date'])
    df = label(df)
    df.to_csv('ml_data/label/' + stock_id + '.csv', index=True, index_label='date')

end_time = time.time()
(end_time - begin_time) / 60

c:\users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


39.87977465391159

In [69]:
#####################
### load datasets
#####################
stock_data = {}
train_dfs = []
for stock_id in db.get_stock_info().index: 
    df = pd.read_csv('ml_data/label/' + stock_id + '.csv', index_col=['date'], parse_dates=['date'])
    stock_data[stock_id] = df
    train_dfs.append(df.iloc[:-INTERVAL])
datasets = pd.concat(train_dfs)
print(datasets.shape)

(50970, 58)


In [70]:
#####################
### feature: label encoding
#####################
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
datasets['industry'] = le.fit_transform(datasets['industry'])

In [88]:
#####################
### over-sampling 
#####################
### weight
begin_time = time.time()

df_extra = pd.DataFrame(columns=df.columns)
for i in range(0, datasets.shape[0]):
    for k in range(1, datasets['weight'].iloc[i]):
        df_extra = df_extra.append(datasets.iloc[i])
df_weight = datasets.append(df_extra).sort_index()
df_weight = df_weight.drop(['weight'], axis=1)
cols = ['industry', 'stock_id', '當月營收_年增次數', '三率升降', '三率升降累積', 'foreign', 'label']
df_weight[cols] = df_weight[cols].applymap(int)
print(df_weight.shape)

end_time = time.time()
print((end_time - begin_time) / 60)

### inbalanced dataset
def balance_label_counts(df_in):
    df_temp = df_in.copy()
    df_class_0 = df_temp[df_temp['label'] == 0]
    count_0 = df_class_0.shape[0]
    df_class_p1 = df_temp[df_temp['label'] == 1]
    count_p1 = df_class_p1.shape[0]
    df_class_m1 = df_temp[df_temp['label'] == -1]
    count_m1 = df_class_m1.shape[0]
    count_max = max(count_0, count_p1, count_m1)

    if count_0 > 0 and count_0 < count_max:
        df_class_0 = df_class_0.sample(count_max, replace=True)
    if count_p1 > 0 and count_p1 < count_max:
        df_class_p1 = df_class_p1.sample(count_max, replace=True)
    if count_m1 > 0 and count_m1 < count_max:
        df_class_m1 = df_class_m1.sample(count_max, replace=True)
    df_all = pd.concat([df_class_0, df_class_p1, df_class_m1]).sort_index()
    print(df_all.shape)
    
    return df_all

(80992, 57)
6.37867108186086


In [72]:
def accuracy_score(s_label, s_predict, silence=False):
    label_buy = np.sum(s_label == 1)
    label_sell = np.sum(s_label == -1)
    predict_buy = np.sum(s_predict == 1)
    predict_sell = np.sum(s_predict == -1)
    if not silence:
        print('buy predict/label :', predict_buy, '/', label_buy, '=', 
              predict_buy / label_buy if label_buy != 0 else None)
        print('sell predict/label :', predict_sell, '/', label_sell, '=', 
              predict_sell / label_sell if label_sell != 0 else None, '\n')
    
    p1_score = []
    m1_score = []
    for label, predict in zip(s_label, s_predict):
        if predict != 0:
            score = 1 if predict == label else 0
            if predict == 1:
                p1_score.append(score)
            else:
                m1_score.append(score)
    p1_score_mean = np.mean(p1_score)
    m1_score_mean = np.mean(m1_score)
    if not silence:
        print('buy true/predict :', np.sum(p1_score), '/', len(p1_score), '=', p1_score_mean)
        print('sell true/predict :', np.sum(m1_score), '/', len(m1_score), '=', m1_score_mean, '\n')
    return (p1_score_mean, m1_score_mean)

In [102]:
#####################
### train
#####################
import numpy as np
import itertools
from sklearn.ensemble import RandomForestClassifier

def build_and_train(df_in, n_estimators=30, max_depth=10, silence=False):
    ### build model
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, max_features='sqrt', random_state=7)
        
    ### split train & test data
    DAY_STEP = INTERVAL
    dates = sorted(list(set(df_in.index)))
    dates_length = len(dates)
    accuracy_test_buy = []
    accuracy_test_sell = []
    
    for i_start in range(DAY_STEP * 3, dates_length, DAY_STEP):
        i_end = i_start + DAY_STEP - 1
        if i_end >= dates_length:
            i_end = dates_length - 1
        if not silence:
            print('==============================')
            print('test data from', dates[i_start].date() , 'to', dates[i_end].date())
        df_train = balance_label_counts(df_in.loc[dates[0] : dates[i_start - 1]])
        df_test = df_in.loc[dates[i_start] : dates[i_end]]
        X_train, X_test = (df_train.drop(['label'], axis=1), df_test.drop(['label'], axis=1))
        y_train, y_test = (df_train.label, df_test.label)
        
        ### train model
        model.fit(X_train, y_train)
    
        y_pred = model.predict(X_train)
        print('Train Result')
        score_buy, score_sell = accuracy_score(y_train, y_pred)

        ### predict
        y_pred = model.predict(X_test)
        print('Test Result')
        score_buy, score_sell = accuracy_score(y_test, y_pred)
        accuracy_test_buy.append(score_buy)
        accuracy_test_sell.append(score_sell)

    print('==============================')
    print('Average Test Accuracy(Buy):', np.mean(accuracy_test_buy))
    print('Average Test Accuracy(Sell):', np.mean(accuracy_test_sell))
    return model

model = build_and_train(df_weight, 30, 8)

test data from 2017-04-25 to 2017-10-17
(53661, 57)
Train Result
buy predict/label : 16561 / 17887 = 0.9258679487896238
sell predict/label : 18609 / 17887 = 1.0403645105383799 

buy true/predict : 14742 / 16561 = 0.890163637461506
sell true/predict : 16224 / 18609 = 0.871836208286313 

Test Result
buy predict/label : 4359 / 6529 = 0.6676366978097718
sell predict/label : 1466 / 1873 = 0.782701548318206 

buy true/predict : 2920 / 4359 = 0.6698784124799266
sell true/predict : 299 / 1466 = 0.20395634379263303 

test data from 2017-10-18 to 2018-04-18
(73248, 57)
Train Result
buy predict/label : 20975 / 24416 = 0.8590678243774574
sell predict/label : 25764 / 24416 = 1.0552096985583224 

buy true/predict : 19201 / 20975 = 0.9154231227651967
sell true/predict : 22347 / 25764 = 0.8673730787144853 

Test Result
buy predict/label : 3893 / 5951 = 0.6541757687783566
sell predict/label : 1705 / 2203 = 0.7739446209714026 

buy true/predict : 2696 / 3893 = 0.6925250449524788
sell true/predict : 635 

In [103]:
feature_importances = pd.Series(data=model.feature_importances_, index=df_weight.columns.delete(-1)).sort_values(ascending=False)
print(feature_importances)

EPS4季            0.058828
close            0.056853
每股淨值             0.046661
DEM_月k           0.043427
毛利率_年增           0.043408
本益比              0.042615
毛利率_季增           0.041133
營益率_年增           0.039387
本淨比              0.037364
12月營收_年增率        0.031003
DIFF_周k          0.030309
industry         0.029987
DEM_周k           0.028742
DIFF_月k          0.028416
EPS4季_季增率        0.027608
stock_id         0.026368
D-M_月k           0.025939
3月營收_年增率         0.025849
D-M_周k           0.025413
close_1-240      0.025412
淨利率_年增           0.025411
眼光費              0.023724
當月營收_年增次數        0.023097
三率升降累積           0.021038
營益率_季增           0.020523
當月營收_年增率         0.019792
D_月k             0.017557
淨利率_季增           0.016640
foreign_20       0.013160
K_月k             0.009460
close_1-60       0.009383
DIFF_日k          0.008924
DEM_日k           0.008136
D_周k             0.007816
K-D_月k           0.007493
三率升降             0.007292
foreign_10       0.006766
K-D_周k           0.005197
K_周k        

In [99]:
from sklearn.externals import joblib
### save model
# joblib.dump(model, 'ml_data/model.pkl')

### load model
# model = joblib.load('ml_data/model.pkl')

In [100]:
#####################
### predict
#####################
# stock_info = db.get_stock_info()
# for stock_id in stock_data:
#     stock_name = stock_info.loc[stock_id, 'stock_name']
#     df_daily_ori = stock_data[stock_id]
#     df_daily = df_daily_ori.copy()
#     df_daily['predict'] = model.predict(df_daily_ori.drop(['label'], axis=1))
#     compare_label(df_daily.label.iloc[:-INTERVAL], df_daily.predict.iloc[:-INTERVAL])
#     print('Test Accuracy', stock_name, stock_id, 
#           accuracy_score(df_daily.label.iloc[:-INTERVAL], df_daily.predict.iloc[:-INTERVAL], False), '\n')

In [101]:
#####################
### plot
#####################
# import matplotlib as mpl
# import matplotlib.pyplot as plt
# mpl.style.use('fivethirtyeight') # fivethirtyeight, fast
# plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] # font to handle chinese
# %matplotlib inline

# def get_region(series, value):
#     region = []
#     get = False
#     for i in range(len(series.index)):
#         if series[i] == value:
#             if not get:
#                 region.append([series.index[i]])
#                 get = True
#             continue
#         if get:
#             region[-1].append(series.index[i - 1])
#             get = False
#     if len(region) > 0 and len(region[-1]) == 1:
#         region[-1].append(series.index[-1])
#     return region

# def draw_region(region, ax, color):
#     for xmin, xmax in region:
#         ax.axvspan(xmin, xmax, color=color, alpha=0.5)
        
# region_buy = get_region(df_daily['predict'], 1)
# ax = df_daily[['close']].plot(grid=True, title=stock_id + ',' + stock_name, figsize=(10,4))
# draw_region(region_buy, ax, 'red')
# region_sell = get_region(df_daily['predict'], -1)
# draw_region(region_sell, ax, 'green')